In [1]:
from torchkf import *
import torch
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import pykeos as pk

pyunicorn: Package netCDF4 could not be loaded. Some functionality in class Data might not be available!
pyunicorn: Package netCDF4 could not be loaded. Some functionality in class NetCDFDictionary might not be available!


In [2]:
level1 = GaussianSystem(
    state_dim=2,
    obs_dim=2,
    fwd_transform=LinearTransform(torch.eye(2)),
    obs_transform=LinearTransform(torch.eye(2)),
    initial_state_mean=torch.zeros(2), 
    initial_state_cov=0.1 * torch.eye(2), 
    process_noise_cov=0.1 * torch.eye(2), 
    obs_noise_cov=0.1 * torch.eye(2),
)

level2 = GaussianSystem(
    state_dim=3,
    obs_dim=2,
    input_dim=2,
    fwd_transform=LinearTransform(torch.randn(3, 3+2)),
    obs_transform=LinearTransform(torch.randn(2, 3)),
    initial_state_mean=torch.zeros(3), 
    initial_state_cov=0.1 * torch.eye(3), 
    process_noise_cov=0.1 * torch.eye(3), 
    obs_noise_cov=0.1 * torch.eye(2),
)

systems = [level1, level2]
hdm = HierarchicalDynamicalModel(systems)

In [3]:
y = torch.Tensor(np.sin(2 * np.pi * np.asarray([[5, 1]]) * np.linspace(0, 5, 500)[:, None] + 2* np.pi *np.random.randint(-1, 1, size=(1,2))))
measurements = y + 0.5* torch.randn(y.shape)

In [4]:
hdm.filter(measurements[None])

Filter: 100%|██████████| 500/500 [00:01<00:00, 386.89it/s]


[[OrderedDict([('x_prev',
                Gaussian(loc: torch.Size([1, 2]), covariance_matrix: torch.Size([1, 2, 2]))),
               ('x_prior',
                Gaussian(loc: torch.Size([1, 2]), covariance_matrix: torch.Size([1, 2, 2]))),
               ('y_prior',
                Gaussian(loc: torch.Size([1, 2]), covariance_matrix: torch.Size([1, 2, 2]))),
               ('u_prior', None),
               ('Px_prev_x_prior',
                tensor([[[0.1000, 0.0000],
                         [0.0000, 0.1000]]])),
               ('Pu_x_prior', None),
               ('Pxy',
                tensor([[[0.2000, 0.0000],
                         [0.0000, 0.2000]]])),
               ('x_post',
                Gaussian(loc: torch.Size([1, 2]), covariance_matrix: torch.Size([1, 2, 2]))),
               ('u_post', None)]),
  OrderedDict([('x_prev',
                Gaussian(loc: torch.Size([1, 3]), covariance_matrix: torch.Size([1, 3, 3]))),
               ('x_prior',
                Gaussian(lo

In [20]:
for system in systems:
    print(system)
    Gaussian(torch.zeros(system['state_dim']),
             system['obs_noise_cov']).expand(torch.Size([1]))

{'state_dim': 2, 'obs_dim': 2, 'input_dim': 0, 'fwd_transform': LinearTransform(), 'obs_transform': LinearTransform(), 'initial_state_mean': tensor([0., 0.]), 'initial_state_cov': tensor([[0.1000, 0.0000],
        [0.0000, 0.1000]]), 'process_noise_cov': tensor([[0.1000, 0.0000],
        [0.0000, 0.1000]]), 'obs_noise_cov': tensor([[0.1000, 0.0000],
        [0.0000, 0.1000]])}
{'state_dim': 3, 'obs_dim': 2, 'input_dim': 2, 'fwd_transform': LinearTransform(), 'obs_transform': LinearTransform(), 'initial_state_mean': tensor([0., 0., 0.]), 'initial_state_cov': tensor([[0.1000, 0.0000, 0.0000],
        [0.0000, 0.1000, 0.0000],
        [0.0000, 0.0000, 0.1000]]), 'process_noise_cov': tensor([[0.1000, 0.0000, 0.0000],
        [0.0000, 0.1000, 0.0000],
        [0.0000, 0.0000, 0.1000]]), 'obs_noise_cov': tensor([[0.1000, 0.0000],
        [0.0000, 0.1000]])}


RuntimeError: The expanded size of the tensor (3) must match the existing size (2) at non-singleton dimension 2.  Target sizes: [1, 3, 3].  Tensor sizes: [2, 2]